<a href="https://colab.research.google.com/github/Rohitsingh24-cloud/pytorch/blob/main/ANNusingpytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [3]:
from  sklearn.model_selection import train_test_split

In [4]:
import pandas as pd

In [5]:
from torch.utils.data import DataLoader,Dataset

In [6]:
import torch.nn as nn

In [8]:
import torch.optim as optim

In [45]:
import matplotlib.pyplot as plt



In [46]:
torch.manual_seed(42)

In [47]:
df=pd.read_csv('fashion-mnist_test.csv')

In [43]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,...,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,...,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,...,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,161,212,138,150,169,164,176,202,255,183,26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,129,221,255,255,215,...,255,255,254,252,255,101,196,254,252,100,0,0,0,0,0,168,232,225,0,41,172,147,148,153,155,146,137,141,143,137,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,204,235,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,69,0,2,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,37,93,72,58,55,63,66,89,159,120,20,0,0,0,0,1,0,0,0,0,0,0,1,0,0,22,67,125,151,168,...,112,115,117,117,113,146,41,0,3,0,0,0,0,0,0,0,1,0,1,76,54,55,51,53,53,55,53,44,38,37,32,23,14,20,0,0,1,0,0,0
9996,6,0,0,0,0,0,0,0,0,0,0,0,76,36,110,170,78,136,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,...,94,84,70,44,0,36,152,65,110,0,0,0,0,0,0,0,21,57,89,7,0,0,0,0,0,0,0,0,0,0,0,0,0,2,52,23,28,0,0,0
9997,8,0,0,0,0,0,0,0,0,0,0,1,3,0,14,152,184,158,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,...,219,223,226,239,247,241,224,213,144,0,0,0,0,0,0,1,0,44,232,199,200,190,180,179,185,189,191,185,179,178,175,172,172,182,199,222,42,0,1,0
9998,8,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,131,192,79,0,0,0,0,...,13,26,17,8,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
#train test split

In [15]:
x=df.iloc[:,1:].values

In [16]:
y=df.iloc[:,0].values

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
x_train=x_train/255.0
x_test=x_test/255.0

In [20]:
#create custom dataset class

In [21]:
class customdataset(Dataset):
  def __init__(self,features,lables):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(lables,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    x=self.features[idx]
    y=self.labels[idx]
    return x,y

In [23]:
train_dataset=customdataset(x_train,y_train)

In [24]:
test_dataset=customdataset(x_test,y_test)

In [25]:
#crate train and test loader

In [26]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)

In [27]:
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [28]:
##define nn class

In [30]:
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model=nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )

    def forward(self,x):
        return self.model(x)


In [31]:
learning_rate=0.1
epochs=100

In [32]:
model=MyNN(x_train.shape[1])

In [33]:
#loss function
criterion=nn.CrossEntropyLoss()

In [34]:
#optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)

In [35]:
#train loop

In [38]:
for epoch in range(epochs):
  total_epoch_loss=0;

  for batch_features,batch_labels in train_loader:
    #forward pass
    outputs=model(batch_features)
    #calculate loss
    Loss=criterion(outputs,batch_labels)
    #back propagate
    optimizer.zero_grad()
    Loss.backward()

    #update gradient
    optimizer.step()
    total_epoch_loss=total_epoch_loss+Loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch:{epoch+1},Loss:{avg_loss}')

Epoch:1,Loss:1.0994572565555571
Epoch:2,Loss:0.6500626981258393
Epoch:3,Loss:0.5457627419233322
Epoch:4,Loss:0.4963847902417183
Epoch:5,Loss:0.4572623638510704
Epoch:6,Loss:0.4235459149479866
Epoch:7,Loss:0.4040894372463226
Epoch:8,Loss:0.3876530373096466
Epoch:9,Loss:0.36599803936481473
Epoch:10,Loss:0.35227703148126605
Epoch:11,Loss:0.33596188634634017
Epoch:12,Loss:0.32192616039514543
Epoch:13,Loss:0.30752689445018766
Epoch:14,Loss:0.30115316703915596
Epoch:15,Loss:0.2904211152791977
Epoch:16,Loss:0.28078760506212713
Epoch:17,Loss:0.2715940577238798
Epoch:18,Loss:0.26088299524784087
Epoch:19,Loss:0.25179853054881096
Epoch:20,Loss:0.2410636202991009
Epoch:21,Loss:0.23085646675527097
Epoch:22,Loss:0.2237499426603317
Epoch:23,Loss:0.2276497804671526
Epoch:24,Loss:0.2196926404759288
Epoch:25,Loss:0.20659015955030918
Epoch:26,Loss:0.1998826839104295
Epoch:27,Loss:0.19209725043922662
Epoch:28,Loss:0.1827486424818635
Epoch:29,Loss:0.17957450166344643
Epoch:30,Loss:0.1713853770196438
Epoch:

In [39]:
#set model to eval

In [41]:
model.eval()
#it tell model that we are going for evaluation

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [42]:
total=0
correct=0;
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    outputs=model(batch_features)
    _,predicted=torch.max(outputs,1)
    total=total+batch_labels.shape[0]
    correct=correct+(predicted==batch_labels).sum().item()
print(correct/total)

0.8405
